In [ ]:

import matplotlib.image as img
import numpy as np
import pandas as pd 
from sklearn.neighbors import KNeighborsRegressor
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from scipy import stats

from PIL import Image

df = pd.read_csv('/content/drive/My Drive/MachineLearning/data/mnist/train.csv')
display(df.shape)

## 3. 데이터 확인해보기 - 어떤 이미지일까?
img_data = df.drop('label', axis=1, inplace=False).values  # 이미지의 픽셀 데이터만 ndarray형식으로
fig = plt.figure()  # 도화지 깔어~
fig_arr = list()  # 나눌 영억을 저장할 리스트

for n in range(10):
    fig_arr.append(fig.add_subplot(2,5,n+1))
    fig_arr[n].imshow(img_data[n].reshape(28,28), cmap='Greys', 
                      interpolation='nearest')

plt.tight_layout()
plt.show()

In [ ]:
x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(df.drop('label', axis=1, inplace=False), 
                 df['label'], test_size=0.3, random_state=0)

## nomalization
scaler = MinMaxScaler()
scaler.fit(x_data_train)
x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)
del x_data_train, x_data_test

x_data_train_norm = x_data_train_norm.reshape((29400, 28, 28, 1))
x_data_test_norm = x_data_test_norm.reshape((12600, 28, 28, 1))

print(x_data_train_norm.shape)
print(x_data_test_norm.shape)
print(t_data_train.shape)
print(t_data_test.shape)

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(Flatten())
model.add(Dropout(rate=0.5))
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()


In [ ]:
# 학습실행
model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])

# model.fit(x_data_train_norm, t_data_train, epochs=5)
history = model.fit(x_data_train_norm,
                    t_data_train,
                    epochs=100,
                    verbose=1,
                    batch_size=100,
                    validation_split=0.3)

In [ ]:
# test_loss, test_acc = model.evaluate(x_data_test_norm,  t_data_test, verbose=2)
# print(test_acc)
result = np.argmax(model.predict(x_data_test_norm), axis=1)
print(classification_report(t_data_test, result))
model.evaluate(x_data_test_norm, t_data_test)

In [ ]:
print(history.history.keys())
plt.plot(history.history['sparse_categorical_accuracy'], color='r')
plt.plot(history.history['val_sparse_categorical_accuracy'], color='b')
plt.show()
plt.plot(history.history['loss'], color='r')
plt.plot(history.history['val_loss'], color='b')
plt.show()